In [1]:
from enums import CollectionNames, EmbeddingModels
from qdrant_client.models import Filter, FieldCondition, MatchValue

from retrieval_util import (
    get_qdrant_client,
    get_embeddings_function,
    retrieve_run_dict_with_documents_with_scores,
    create_llm_for_self_query_retriever
)

import os
import json
import logging
from ranx import Qrels, Run, evaluate, compare

os.environ[
    "TOKENIZERS_PARALLELISM"] = "false"  # Verhindert Fehler beim Erzeugen von Embeddings (Tritt sporadisch auf, konnte ich nicht gezielt reproduzieren)
logging.basicConfig(level=logging.INFO)

**use_original_query=False** gesetzt? -> In retrieval_util.py setzen! Siehe kommentar im unteren Feld

Nach Änderung muss der Jupyter Server neu gestartet werden, damit die Änderungen wirksam werden.

In [2]:
# in der retrieval_util.py muss der Parameter use_original_query=False gesetzt werden (in der funktion für den SelfQueryRetriever)
# Behilfslösung, da ich nicht die Signatur der Funktion umschreiben will, sonst muss das beim Aufruf in den vorherigen Evaluationen auch wieder angepasst werden
SELF_QUERY_RETRIEVER_FRAGEN = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}
'''
Richtig vergleichbar ist dieser Ansatz nicht, da die Fragen nicht identisch sind.
Ebenso werden die vereinfachten Fragen nicht mit den selbst formulierten übereinstimmen, da sie durch das LLM generiert werden.

Hiermit soll dennoch gezeigt werden, wie dieser Ansatz performt.
Für ein reales System eignet sich der SelfQueryRetriever potenziell nicht.
Wenn in einem erweiterten System weitere Felder für das Filtern (z. B. Studienmodelle) hinzugefügt werden,
müssten auch diese in der Fragestellung eingebunden werden. Die Fragen würden immer länger und komplexer werden.
Das ist aber ein Aspekt der Gebrauchstauglichkeit und nicht der Funktionsweise/Performance des Systems.

Daher wird der Ansatz zwar mit evaluiert, in der Arbeit schließe ich diesen jedoch aufgrund der genannten Aspekte aus.
'''

'\nRichtig vergleichbar ist dieser Ansatz nicht, da die Fragen nicht identisch sind.\nEbenso werden die vereinfachten Fragen nicht mit den selbst formulierten übereinstimmen, da sie durch das LLM generiert werden.\n\nHiermit soll dennoch gezeigt werden, wie dieser Ansatz performt.\nFür ein reales System eignet sich der SelfQueryRetriever potenziell nicht.\nWenn in einem erweiterten System weitere Felder für das Filtern (z. B. Studienmodelle) hinzugefügt werden,\nmüssten auch diese in der Fragestellung eingebunden werden. Die Fragen würden immer länger und komplexer werden.\nDas ist aber ein Aspekt der Gebrauchstauglichkeit und nicht der Funktionsweise/Performance des Systems.\n\nDaher wird der Ansatz zwar mit evaluiert, in der Arbeit schließe ich diesen jedoch aufgrund der genannten Aspekte aus.\n'

Um die Kennzahlen @10 und @5 zu berechnen, muss die Konstante MAX_DOCUMENTS auf den entsprechenden Wert gesetzt werden.

Außerdem muss der BASE_PATH angepasst werden, um die Ergebnisse zu speichern.

Zuletzt muss bei den Metriken @5 oder @10 gesetzt werden.

In [3]:
# Konstanten
MAX_DOCUMENTS = 10
BASE_PATH = "./mixedbread_retrieval_results_AT_10"
QDRANT_URL = "http://localhost:6333"
LLM = create_llm_for_self_query_retriever()
FRAGEN = {
    1: "Wann kann ich mich einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit schreiben?",
    7: "Wie sind Portfolioprüfungen aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit?",
    10: "Wie lange dauern die Klausuren?"
}
METRICS = ["ndcg@10", "mrr@10", "map@10", "precision@10", "recall@10", "hits@10", "hit_rate@10"]


# Alle relevanten Filter für die Fragen
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )


FILTERS = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

In [5]:
embeddings = get_embeddings_function(model_name=EmbeddingModels.MIXEDBREAD.value)


2025-03-18 08:44:35,199 [INFO] Use pytorch device_name: mps
2025-03-18 08:44:35,199 [INFO] Load pretrained SentenceTransformer: mixedbread-ai/deepset-mxbai-embed-de-large-v1


In [6]:
token_based_client = get_qdrant_client(embeddings=embeddings,
                                       collection_name=CollectionNames.MIXEDBREAD_TOKEN_BASED.value,
                                       qdrant_url=QDRANT_URL)

recursive_client = get_qdrant_client(embeddings=embeddings, collection_name=CollectionNames.MIXEDBREAD_RECURSIVE.value,
                                     qdrant_url=QDRANT_URL)

2025-03-18 08:44:44,378 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-18 08:44:44,386 [INFO] HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks "HTTP/1.1 200 OK"
2025-03-18 08:44:44,712 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-18 08:44:44,718 [INFO] HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks "HTTP/1.1 200 OK"


In [10]:
result = token_based_client.similarity_search(query=FRAGEN[1], k=10, filter=FILTERS[1])

2025-03-18 08:47:11,811 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


In [12]:
for fragen_id, frage in FRAGEN.items():
    print(f"{fragen_id}: {frage}")
    result = token_based_client.similarity_search(query=frage, k=MAX_DOCUMENTS, filter=FILTERS[fragen_id])

    for r in result:
        print(r.metadata["id"])

1: Wann kann ich mich einschreiben?


2025-03-18 08:48:39,982 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,097 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


1021
1025
1047
1020
1033
1028
1041
1026
1027
1022
2: Welche Literatur ist relevant für das Modul Mathematik?
1264
1394
1382
1283
1262
1293
1472
1297
1416
1320
3: Welche Fachgebiete sind an der FH vertreten?


2025-03-18 08:48:40,207 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,249 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,294 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,345 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,391 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


1481
1486
1479
1476
1478
1489
1480
1544
1487
1503
4: Ist eine Beurlaubung während des Studiums möglich?
1499
1500
1554
1555
1521
1502
1553
1482
1498
1543
5: Welche Studienmodelle werden angeboten?
1482
1486
1483
1480
1484
1479
1485
1543
1494
1505
6: Wie viele Seiten muss ich in meiner Bachelorarbeit schreiben?
193
19
122
192
15
16
18
76
191
194
7: Wie sind Portfolioprüfungen aufgebaut?
1192
1464
1193
1405
1191
1378
1269
1414
1189
1402
8: Was muss ich beachten, wenn ich bei einer Prüfung krank bin?


2025-03-18 08:48:40,492 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,586 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-18 08:48:40,631 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


1555
1554
1553
1530
1535
1499
1534
1521
1539
1543
9: Was sind die Inhalte im Modul IT-Sicherheit?
704
705
675
747
563
703
690
573
558
650
10: Wie lange dauern die Klausuren?
14
15
152
145
77
106
217
82
96
133


Zeigt, dass die Reihenfolge dieselbe ist, wie mit score.